In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [13]:
!pip install gdown
!gdown 13WRuCCpUFPSqwQ3csIe7-UcoSfVcHjpU
!gdown 1Gnay_fDFnaB7U9PcTZEgv24RcnywhD0_
!gdown 1GUbdSU-ls_THuU-LStkFR6nA3dEdaG2X 

Downloading...
From: https://drive.google.com/uc?id=13WRuCCpUFPSqwQ3csIe7-UcoSfVcHjpU
To: /kaggle/working/data_train_legal_qa_new.json
100%|██████████████████████████████████████| 5.83M/5.83M [00:00<00:00, 51.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Gnay_fDFnaB7U9PcTZEgv24RcnywhD0_
To: /kaggle/working/alqac25_private_test_task2.json
100%|██████████████████████████████████████| 61.0k/61.0k [00:00<00:00, 80.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GUbdSU-ls_THuU-LStkFR6nA3dEdaG2X
To: /kaggle/working/alqac25_law.json
100%|███████████████████████████████████████| 5.21M/5.21M [00:00<00:00, 104MB/s]


## load data train

In [3]:
import json
import random

random.seed(42)

# Giả định đường dẫn tệp JSON
file_path = 'data_train_legal_qa_new.json'

# Bước 1: Tải dữ liệu
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        questions = json.load(f)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp {file_path}. Vui lòng đảm bảo tệp tồn tại và đúng đường dẫn.")
    exit()
except json.JSONDecodeError:
    print(f"Lỗi: Không thể giải mã JSON từ tệp {file_path}. Đảm bảo tệp có định dạng JSON hợp lệ.")
    exit()

# Bước 2: Phân loại câu hỏi
true_false_questions = [q for q in questions if q['question_type'] == 'Đúng/Sai']
multiple_choice_questions = [q for q in questions if q['question_type'] == 'Trắc nghiệm']
essay_questions = [q for q in questions if q['question_type'] == 'Tự luận']

# Số lượng ban đầu của từng loại
num_true_false_total = len(true_false_questions)
num_multiple_choice_total = len(multiple_choice_questions)
num_essay_total = len(essay_questions)
total_samples = len(questions)

print(f"Tổng số mẫu ban đầu: {total_samples}")
print(f" - Đúng/Sai: {num_true_false_total}")
print(f" - Trắc nghiệm: {num_multiple_choice_total}")
print(f" - Tự luận: {num_essay_total}\n")

# Bước 3: Chuẩn bị chia tập dữ liệu
validation_set_target_size = 100
num_essay_in_val_requested = 5

validation_questions = []
train_questions = []

# --- Xử lý câu hỏi 'Tự luận' cho tập validation ---
if num_essay_total < num_essay_in_val_requested:
    print(f"Cảnh báo: Số lượng câu hỏi 'Tự luận' ({num_essay_total}) ít hơn số lượng yêu cầu cho tập validation ({num_essay_in_val_requested}). Sẽ lấy tất cả {num_essay_total} câu hỏi 'Tự luận' cho tập validation.")
    num_essay_in_val_actual = num_essay_total
else:
    num_essay_in_val_actual = num_essay_in_val_requested

random.shuffle(essay_questions)
val_essay_q = essay_questions[:num_essay_in_val_actual]
train_essay_q = essay_questions[num_essay_in_val_actual:]

validation_questions.extend(val_essay_q)
train_questions.extend(train_essay_q)

# --- Xử lý câu hỏi 'Đúng/Sai' và 'Trắc nghiệm' cho phần còn lại của tập validation ---
remaining_val_slots = validation_set_target_size - len(val_essay_q)

if remaining_val_slots > 0:
    total_tf_mc_available = num_true_false_total + num_multiple_choice_total

    if total_tf_mc_available == 0:
        print("Không có câu hỏi 'Đúng/Sai' hoặc 'Trắc nghiệm' nào trong dữ liệu gốc để chia cho tập validation.")
    else:
        # Tính toán tỷ lệ dựa trên tổng số câu hỏi Đúng/Sai và Trắc nghiệm ban đầu
        prop_tf = num_true_false_total / total_tf_mc_available
        prop_mc = num_multiple_choice_total / total_tf_mc_available

        # Tính số lượng mong muốn cho tập validation dựa trên tỷ lệ và số slot còn lại
        num_tf_val_desired = round(remaining_val_slots * prop_tf)
        num_mc_val_desired = round(remaining_val_slots * prop_mc)
        
        # Điều chỉnh làm tròn để tổng số lượng chính xác với remaining_val_slots
        current_sum_desired = num_tf_val_desired + num_mc_val_desired
        diff = remaining_val_slots - current_sum_desired
        if diff != 0:
            if diff > 0: # Cần thêm mẫu
                # Ưu tiên thêm vào loại có tỷ lệ lớn hơn để duy trì tính đại diện
                if prop_tf >= prop_mc:
                    num_tf_val_desired += diff
                else:
                    num_mc_val_desired += diff
            else: # Cần bớt mẫu (diff là số âm)
                # Ưu tiên bớt từ loại có tỷ lệ lớn hơn
                if prop_tf >= prop_mc:
                    num_tf_val_desired += diff # diff là số âm
                else:
                    num_mc_val_desired += diff # diff là số âm
        
        # Đảm bảo không lấy quá số lượng có sẵn trong dữ liệu gốc
        num_tf_val_final = min(num_tf_val_desired, num_true_false_total)
        num_mc_val_final = min(num_mc_val_desired, num_multiple_choice_total)

        # Kiểm tra xem tổng số lượng sau khi capping có đủ để điền vào remaining_val_slots không
        if num_tf_val_final + num_mc_val_final < remaining_val_slots:
            print(f"Cảnh báo: Không đủ câu hỏi Đúng/Sai và Trắc nghiệm để điền đầy đủ {remaining_val_slots} vị trí còn lại trong tập validation. Sẽ lấy tất cả {num_tf_val_final + num_mc_val_final} câu hỏi Đúng/Sai và Trắc nghiệm có thể.")
        
        random.shuffle(true_false_questions)
        random.shuffle(multiple_choice_questions)

        val_tf_q = true_false_questions[:num_tf_val_final]
        val_mc_q = multiple_choice_questions[:num_mc_val_final]

        train_tf_q = true_false_questions[num_tf_val_final:]
        train_mc_q = multiple_choice_questions[num_mc_val_final:]

        validation_questions.extend(val_tf_q)
        validation_questions.extend(val_mc_q)

        train_questions.extend(train_tf_q)
        train_questions.extend(train_mc_q)
else:
    print("Không còn chỗ trống trong tập validation để thêm câu hỏi Đúng/Sai hoặc Trắc nghiệm (đã đủ 100 mẫu từ 'Tự luận').")


# Bước 4: Kiểm tra và in kết quả
num_train_true_false = sum(1 for q in train_questions if q['question_type'] == 'Đúng/Sai')
num_train_multiple_choice = sum(1 for q in train_questions if q['question_type'] == 'Trắc nghiệm')
num_train_essay = sum(1 for q in train_questions if q['question_type'] == 'Tự luận')
total_train = len(train_questions)

num_val_true_false = sum(1 for q in validation_questions if q['question_type'] == 'Đúng/Sai')
num_val_multiple_choice = sum(1 for q in validation_questions if q['question_type'] == 'Trắc nghiệm')
num_val_essay = sum(1 for q in validation_questions if q['question_type'] == 'Tự luận')

print("\n--- Kết quả chia tập dữ liệu ---")
print(f"Tổng số mẫu tập huấn luyện (train): {total_train}")
print(f" - Đúng/Sai: {num_train_true_false}")
print(f" - Trắc nghiệm: {num_train_multiple_choice}")
print(f" - Tự luận: {num_train_essay}\n")

print(f"Tổng số mẫu tập kiểm định (valid): {len(validation_questions)}")
print(f" - Đúng/Sai: {num_val_true_false}")
print(f" - Trắc nghiệm: {num_val_multiple_choice}")
print(f" - Tự luận: {num_val_essay}\n")

print(f"Tổng cộng: {total_train + len(validation_questions)} mẫu (khớp với {total_samples} ban đầu).")

Tổng số mẫu ban đầu: 728
 - Đúng/Sai: 387
 - Trắc nghiệm: 285
 - Tự luận: 56


--- Kết quả chia tập dữ liệu ---
Tổng số mẫu tập huấn luyện (train): 628
 - Đúng/Sai: 332
 - Trắc nghiệm: 245
 - Tự luận: 51

Tổng số mẫu tập kiểm định (valid): 100
 - Đúng/Sai: 55
 - Trắc nghiệm: 40
 - Tự luận: 5

Tổng cộng: 728 mẫu (khớp với 728 ban đầu).


In [4]:
from datasets import Dataset
train_dataset = Dataset.from_list(train_questions)
valid_dataset = Dataset.from_list(validation_questions)

In [5]:
# Chuyển đổi định dạng dataset sang 'messages' nếu bạn muốn fine-tune với chat_template
# Điều này rất hữu ích cho các mô hình như Qwen, DeepSeek-R1-Distill-Qwen2.5-7B
def format_to_chat_template(example):
    messages = [
        {"role": "system", "content": example["system_prompt"]},
        {"role": "user", "content": example["prompt"]},
        {"role": "assistant", "content": example["answer_think"]},
    ]
    # Unsloth sẽ tự động áp dụng tokenizer.apply_chat_template khi bạn huấn luyện
    # Tuy nhiên, nếu bạn muốn kiểm tra trước, bạn có thể gọi tokenizer.apply_chat_template(messages, tokenize=False)
    return {"messages": messages}

# Áp dụng hàm chuyển đổi cho toàn bộ dataset
train_dataset = train_dataset.map(format_to_chat_template) #remove_columns=['question_id', 'text', 'relevant_articles', 'system_prompt', 'prompt', 'answer', 'answer_think', 'question_type'])
valid_dataset = valid_dataset.map(format_to_chat_template) #remove_columns=['question_id', 'text', 'relevant_articles', 'system_prompt', 'prompt', 'answer', 'answer_think', 'question_type'])

Map:   0%|          | 0/628 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
print("\nMột mẫu dữ liệu sau khi được định dạng lại cho chat_template:")
print(valid_dataset[0])
print("\nCác cột trong dataset sau khi xử lý:")
print(valid_dataset.column_names)


Một mẫu dữ liệu sau khi được định dạng lại cho chat_template:
{'question_id': 'train_alqac25_400', 'question_type': 'Tự luận', 'text': 'Ai là người đứng đầu Nhà nước, thay mặt nước Cộng hòa xã hội chủ nghĩa Việt Nam về đối nội và đối ngoại?', 'relevant_articles': ['Chủ tịch nước là người đứng đầu Nhà nước, thay mặt nước Cộng hoà xã hội chủ nghĩa Việt Nam về đối nội và đối ngoại.'], 'answer': 'chủ tịch nước', 'system_prompt': 'Bạn là một trợ lý pháp lý Tiếng Việt, có nhiệm vụ trả lời các câu hỏi ngắn gọn, chính xác, dựa trên nội dung điều luật được cung cấp. Chỉ trả lời bằng Tiếng Việt.\n\nBạn hãy trả lời theo định dạng sau:\n<think>\n[Suy nghĩ, phân tích của bạn]\n</think>\n[Câu trả lời của bạn]\n', 'prompt': 'Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.\n\nLoại câu hỏi: Tự luận\n\nCâu hỏi: Ai là người đứng đầu Nhà nước, thay mặt nước Cộng hòa xã hội chủ nghĩa Việt Nam về đối nội và đối ngoại?\n\nBối cảnh: \nChủ tịch nước là người đứng đầu Nhà nước, thay mặt 

## load data test

In [22]:
import json

# Load data
with open('alqac25_private_test_task2.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

with open('alqac25_law.json', 'r', encoding='utf-8') as f:
    corpus = json.load(f)

#Build lookup dictionary: { law_id: { article_id: text } }
corpus_dict = {}
for law in corpus:
    law_id = law['id']
    articles_dict = {article['id']: article['text'] for article in law.get('articles', [])}
    corpus_dict[law_id] = articles_dict

# Replace relevant_articles in questions
for q in questions:
    new_articles = []
    for ref in q.get('relevant_articles', []):
        law_id = ref['law_id']
        article_id = ref['article_id']
        text = corpus_dict.get(law_id, {}).get(article_id)
        if text:
            new_articles.append(text)
        else:
            print('error')
    q['relevant_articles'] = new_articles  # Replace with list of strings

# Save the updated file
with open('alqac25_private_test_task2_question_text.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=2)

In [23]:
import json
with open('alqac25_private_test_task2_question_text.json', 'r', encoding='utf-8') as f:
    test_questions = json.load(f)
test_questions[0]

{'question_id': 'private_test_alquac25_1',
 'question_type': 'Đúng/Sai',
 'text': 'Hợp đồng điện tử được ký kết giữa hai hệ thống thông tin tự động mà không có sự can thiệp của con người sẽ không có giá trị pháp lý trong mọi trường hợp.',
 'relevant_articles': ['Hợp đồng điện tử\n1. Hợp đồng điện tử được giao kết hoặc thực hiện từ sự tương tác giữa một hệ thống thông tin tự động với người hoặc giữa các hệ thống thông tin tự động với nhau không bị phủ nhận giá trị pháp lý chỉ vì không có sự kiểm tra hay can thiệp của con người vào từng hành động cụ thể do các hệ thống thông tin tự động thực hiện hay vào hợp đồng.\n2. Bộ trưởng, Thủ trưởng cơ quan ngang Bộ ban hành theo thẩm quyền hoặc trình cấp có thẩm quyền ban hành quy định về giao kết và thực hiện hợp đồng điện tử trong lĩnh vực thuộc phạm vi nhiệm vụ, quyền hạn được phân công, phù hợp với điều kiện thực tiễn.']}

In [24]:
system_prompt = """Bạn là một trợ lý pháp lý Tiếng Việt, có nhiệm vụ trả lời các câu hỏi ngắn gọn, chính xác, dựa trên nội dung điều luật được cung cấp. Chỉ trả lời bằng Tiếng Việt.

Bạn hãy trả lời theo định dạng sau:
<think>
[Suy nghĩ, phân tích của bạn]
</think>
[Câu trả lời của bạn]
"""
def build_prompt(question: dict):
    question_text, article_texts, question_type = question['text'], question['relevant_articles'], question['question_type']
    texts = [a for a in article_texts if isinstance(a, str)]
    article_content = "\n\n".join(texts)
    if question_type == "Tự luận":
        prompt = (
            "Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.\n\n"
            f"Loại câu hỏi: Tự luận\n\n"
            f"Câu hỏi: {question_text}\n\n"
            f"Bối cảnh: \n{article_content}\n\n"
            "Hãy sinh phần suy luận chi tiết theo mẫu bên dưới trong thẻ <think>...</think>. "
            "Bạn cần viết đầy đủ các bước phân tích, dẫn chứng và suy luận trước khi đưa ra câu trả lời ngắn gọn.\n"
            "Không được trả lời ngay mà phải suy luận đầy đủ trước trong <think>.\n\n"
            "<think>\n"
            "1. Phân tích câu hỏi: [trình bày ngắn gọn nội dung và ý định của câu hỏi]\n"
            "2. Dẫn chứng từ bối cảnh:\n"
            "   - Hãy tách từng đoạn dài trong bối cảnh thành nhiều ý nhỏ rõ ràng.\n"
            "   - Hãy tách ít nhất 3 đến 5 ý ở phần dẫn chứng từ bối cảnh\n"
            "   - Mỗi ý nên nêu rõ nội dung pháp lý, viết ngắn gọn dễ hiểu.\n"
            "   - Ví dụ:\n"
            "       - [ý 1 từ đoạn luật A]\n"
            "       - [ý 2 từ đoạn luật A]\n"
            "       - [ý 3 từ đoạn luật B]\n"
            "   - Ghi rõ đoạn nào có liên quan đến câu hỏi.\n"
            "3. Suy luận step-by-step:\n"
            "   a) [bước suy luận 1 dựa trên dẫn chứng ở trên]\n"
            "   b) [bước suy luận 2 tiếp theo]\n"
            "   …\n"
            "4. Kết luận: [tóm tắt câu trả lời cuối cùng dựa trên suy luận]\n"
            "</think>\n\n"
            "[Kết luận cuối cùng sau khi suy luận]"
        )
    elif question_type == "Trắc nghiệm":
        prompt = (
            "Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.\n\n"
            f"Loại câu hỏi: Trắc nghiệm (format của Kết luận cuối cùng sau khi suy luận là 1 trong 4 kết luận: 'A', 'B', 'C', 'D'. Không được giải thích gì thêm.)\n\n"
            f"Câu hỏi: {question_text}\n\n"
            f"Bối cảnh: \n{article_content}\n\n"
            f"4 lựa chọn: \n"
            f"A: {q['choices']['A']}\n"
            f"B: {q['choices']['B']}\n"
            f"C: {q['choices']['C']}\n"
            f"D: {q['choices']['D']}\n"
            "\n\n"
            "Hãy sinh phần suy luận chi tiết theo mẫu bên dưới trong thẻ <think>...</think>. "
            "Bạn cần viết đầy đủ các bước phân tích, dẫn chứng và suy luận trước khi đưa ra câu trả lời ngắn gọn.\n"
            "Không được trả lời ngay mà phải suy luận đầy đủ trước trong <think>.\n\n"
            "<think>\n"
            "1. Phân tích câu hỏi: [trình bày ngắn gọn nội dung và ý định của câu hỏi]\n"
            "2. Dẫn chứng từ bối cảnh:\n"
            "   - Hãy tách từng đoạn dài trong bối cảnh thành nhiều ý nhỏ rõ ràng.\n"
            "   - Hãy tách ít nhất 3 đến 5 ý ở phần dẫn chứng từ bối cảnh\n"
            "   - Mỗi ý nên nêu rõ nội dung pháp lý, viết ngắn gọn dễ hiểu.\n"
            "   - Ví dụ:\n"
            "       - [ý 1 từ đoạn luật A]\n"
            "       - [ý 2 từ đoạn luật A]\n"
            "       - [ý 3 từ đoạn luật B]\n"
            "   - Ghi rõ đoạn nào có liên quan đến câu hỏi.\n"
            "3. Suy luận step-by-step:\n"
            "   a) [bước suy luận 1 dựa trên dẫn chứng ở trên]\n"
            "   b) [bước suy luận 2 tiếp theo]\n"
            "   …\n"
            "4. Kết luận: [tóm tắt câu trả lời cuối cùng dựa trên suy luận]\n"
            "</think>\n\n"
            "[Kết luận cuối cùng sau khi suy luận]"
        )
    elif question_type == "Đúng/Sai":
            prompt = (
            "Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.\n\n"
            f"Loại câu hỏi: Đúng/Sai (format của Kết luận cuối cùng sau khi suy luận là 1 trong 2 kết luận: 'Đúng', 'Sai'. Không được giải thích gì thêm.)\n\n"
            f"Câu hỏi: {question_text}\n\n"
            f"Bối cảnh: \n{article_content}\n\n"
            "Hãy sinh phần suy luận chi tiết theo mẫu bên dưới trong thẻ <think>...</think>. "
            "Bạn cần viết đầy đủ các bước phân tích, dẫn chứng và suy luận trước khi đưa ra câu trả lời ngắn gọn.\n"
            "Không được trả lời ngay mà phải suy luận đầy đủ trước trong <think>.\n\n"
            "<think>\n"
            "1. Phân tích câu hỏi: [trình bày ngắn gọn nội dung và ý định của câu hỏi]\n"
            "2. Dẫn chứng từ bối cảnh:\n"
            "   - Hãy tách từng đoạn dài trong bối cảnh thành nhiều ý nhỏ rõ ràng.\n"
            "   - Hãy tách ít nhất 3 đến 5 ý ở phần dẫn chứng từ bối cảnh\n"
            "   - Mỗi ý nên nêu rõ nội dung pháp lý, viết ngắn gọn dễ hiểu.\n"
            "   - Ví dụ:\n"
            "       - [ý 1 từ đoạn luật A]\n"
            "       - [ý 2 từ đoạn luật A]\n"
            "       - [ý 3 từ đoạn luật B]\n"
            "   - Ghi rõ đoạn nào có liên quan đến câu hỏi.\n"
            "3. Suy luận step-by-step:\n"
            "   a) [bước suy luận 1 dựa trên dẫn chứng ở trên]\n"
            "   b) [bước suy luận 2 tiếp theo]\n"
            "   …\n"
            "4. Kết luận: [tóm tắt câu trả lời cuối cùng dựa trên suy luận]\n"
            "</think>\n\n"
            "[Kết luận cuối cùng sau khi suy luận]"
        )
    else:
        print('Lỗi')

    return prompt

for q in questions:
    q['system_prompt'] = system_prompt
    q['prompt'] = build_prompt(q)

with open('alqac25_private_test_task2_question_text_new.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=2)

In [26]:
with open('alqac25_private_test_task2_question_text_new.json', 'r', encoding='utf-8') as f:
    test_questions = json.load(f)
test_questions[0]

{'question_id': 'private_test_alquac25_1',
 'question_type': 'Đúng/Sai',
 'text': 'Hợp đồng điện tử được ký kết giữa hai hệ thống thông tin tự động mà không có sự can thiệp của con người sẽ không có giá trị pháp lý trong mọi trường hợp.',
 'relevant_articles': ['Hợp đồng điện tử\n1. Hợp đồng điện tử được giao kết hoặc thực hiện từ sự tương tác giữa một hệ thống thông tin tự động với người hoặc giữa các hệ thống thông tin tự động với nhau không bị phủ nhận giá trị pháp lý chỉ vì không có sự kiểm tra hay can thiệp của con người vào từng hành động cụ thể do các hệ thống thông tin tự động thực hiện hay vào hợp đồng.\n2. Bộ trưởng, Thủ trưởng cơ quan ngang Bộ ban hành theo thẩm quyền hoặc trình cấp có thẩm quyền ban hành quy định về giao kết và thực hiện hợp đồng điện tử trong lĩnh vực thuộc phạm vi nhiệm vụ, quyền hạn được phân công, phù hợp với điều kiện thực tiễn.'],
 'system_prompt': 'Bạn là một trợ lý pháp lý Tiếng Việt, có nhiệm vụ trả lời các câu hỏi ngắn gọn, chính xác, dựa trên nội

In [29]:
from datasets import Dataset
test_dataset = Dataset.from_list(test_questions)

In [30]:
# Chuyển đổi định dạng dataset sang 'messages' nếu bạn muốn fine-tune với chat_template
# Điều này rất hữu ích cho các mô hình như Qwen, DeepSeek-R1-Distill-Qwen2.5-7B
def format_to_chat_template(example):
    messages = [
        {"role": "system", "content": example["system_prompt"]},
        {"role": "user", "content": example["prompt"]},
    ]
    # Unsloth sẽ tự động áp dụng tokenizer.apply_chat_template khi bạn huấn luyện
    # Tuy nhiên, nếu bạn muốn kiểm tra trước, bạn có thể gọi tokenizer.apply_chat_template(messages, tokenize=False)
    return {"messages": messages}

# Áp dụng hàm chuyển đổi cho toàn bộ dataset
test_dataset = test_dataset.map(format_to_chat_template) #remove_columns=['question_id', 'text', 'relevant_articles', 'system_prompt', 'prompt', 'answer', 'answer_think', 'question_type'])

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

In [31]:
print("\nMột mẫu dữ liệu sau khi được định dạng lại cho chat_template:")
print(test_dataset[0])
print("\nCác cột trong test dataset sau khi xử lý:")
print(test_dataset.column_names)


Một mẫu dữ liệu sau khi được định dạng lại cho chat_template:
{'question_id': 'private_test_alquac25_1', 'question_type': 'Đúng/Sai', 'text': 'Hợp đồng điện tử được ký kết giữa hai hệ thống thông tin tự động mà không có sự can thiệp của con người sẽ không có giá trị pháp lý trong mọi trường hợp.', 'relevant_articles': ['Hợp đồng điện tử\n1. Hợp đồng điện tử được giao kết hoặc thực hiện từ sự tương tác giữa một hệ thống thông tin tự động với người hoặc giữa các hệ thống thông tin tự động với nhau không bị phủ nhận giá trị pháp lý chỉ vì không có sự kiểm tra hay can thiệp của con người vào từng hành động cụ thể do các hệ thống thông tin tự động thực hiện hay vào hợp đồng.\n2. Bộ trưởng, Thủ trưởng cơ quan ngang Bộ ban hành theo thẩm quyền hoặc trình cấp có thẩm quyền ban hành quy định về giao kết và thực hiện hợp đồng điện tử trong lĩnh vực thuộc phạm vi nhiệm vụ, quyền hạn được phân công, phù hợp với điều kiện thực tiễn.'], 'system_prompt': 'Bạn là một trợ lý pháp lý Tiếng Việt, có nhi

## load model

In [32]:
from unsloth import FastLanguageModel
import torch

# max_seq_length = 10000 # set cao hơn khi train?
max_seq_length = 8192 
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lmq1909/Qwen3-8B-LQA-14e-GRPO-100s-save1-full",
    max_seq_length = max_seq_length,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-21 13:31:03.607415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753104663.953483      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753104664.053917      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


==((====))==  Unsloth 2025.7.6: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [33]:
# def generate_think_response(messages: list[dict]) -> str: 
#     text = tokenizer.apply_chat_template(
#         messages,
#         tokenize = False,
#         add_generation_prompt = True, # Must add for generation
#         enable_thinking = True, # không cần trong mô hình này
#     )
#     max_input_tokens = len(tokenizer(prompt)["input_ids"])
#     max_new_tokens = max_seq_length - max_input_tokens
    
#     generated_output = model.generate(
#         **tokenizer(text, return_tensors = "pt").to("cuda"),
#         max_new_tokens = max_new_tokens,
#         temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
#     )
    
#     # Giải mã token ID thành chuỗi văn bản
#     decoded_output = tokenizer.decode(generated_output[0], skip_special_tokens=True)
#     response_text = decoded_output.split('<｜Assistant｜>')[-1]
#     return response_text
def generate_think_response(messages: list[dict], question_id: str = "") -> str:
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True,
    )

    max_new_tokens = 512
    max_retry = 4

    for attempt in range(max_retry):
        try:
            inputs = tokenizer(text, return_tensors="pt")
            inputs = {k: v.to("cuda") for k, v in inputs.items()}

            generated_output = model.generate(
                **inputs,
                max_new_tokens=int(max_new_tokens),
                temperature=0.6, top_p=0.95, top_k=20,
            )

            decoded_output = tokenizer.decode(generated_output[0], skip_special_tokens=True)
            response_text = decoded_output.split('<｜Assistant｜>')[-1]

            # print('response_text', response_text, '\n\n')
            if "kết luận" in response_text.lower():
                return response_text
            else:
                max_new_tokens *= 2
                print(f"Tăng max_new_tokens lên {max_new_tokens} cho question_id = {question_id}")

        except Exception as e:
            print(f"[ERROR] Lỗi sinh văn bản cho câu hỏi {question_id}: {e}")
            break

    # ❌ Nếu sau 3 lần vẫn không có end token
    print(f"Lỗi generate_think_response cho question_id = {question_id}")
    return response_text

In [34]:
system_prompt = """Bạn là một trợ lý pháp lý Tiếng Việt, có nhiệm vụ trả lời các câu hỏi ngắn gọn, chính xác, dựa trên nội dung điều luật được cung cấp. Chỉ trả lời bằng Tiếng Việt.

Bạn hãy trả lời theo định dạng sau:
<think>
[Suy nghĩ, phân tích của bạn]
</think>
[Câu trả lời của bạn]
"""
prompt = '''Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.

Loại câu hỏi: Trắc nghiệm (format của Kết luận cuối cùng sau khi suy luận là 1 trong 4 kết luận: 'A', 'B', 'C', 'D'. Không được giải thích gì thêm.)

Câu hỏi: Chồng không có quyền yêu cầu ly hôn 
trong trường hợp nào ?

Bối cảnh: 
Quyền yêu cầu giải quyết ly hôn
1. Vợ, chồng hoặc cả hai người có quyền yêu cầu Tòa án giải quyết ly hôn.

2. Cha, mẹ, người thân thích khác có quyền yêu cầu Tòa án giải quyết ly hôn khi một bên vợ, chồng do bị bệnh tâm thần hoặc mắc bệnh khác mà không thể nhận thức, làm chủ được hành vi của mình, đồng thời là nạn nhân của bạo lực gia đình do chồng, vợ của họ gây ra làm ảnh hưởng nghiêm trọng đến tính mạng, sức khỏe, tinh thần của họ.

3. Chồng không có quyền yêu cầu ly hôn trong trường hợp vợ đang có thai, sinh con hoặc đang nuôi con dưới 12 tháng tuổi.

4 lựa chọn: 
A: Vợ bị bệnh tâm thần, không thể nhận thức, làm chủ hành vi của mình.
B:Vợ ngoại tình với người đàn ông khác.
C: Vợ đang có thai, sinh con hoặc đang 
nuôi con dưới 12 tháng tuổi.
D:Hai vợ chồng không hợp nhau, thường xuyên xảy ra xung đột, cãi vã. 


Hãy sinh phần suy luận chi tiết theo mẫu bên dưới trong thẻ <think>...</think>. Bạn cần viết đầy đủ các bước phân tích, dẫn chứng và suy luận trước khi đưa ra câu trả lời ngắn gọn.
Không được trả lời ngay mà phải suy luận đầy đủ trước trong <think>.

<think>
1. Phân tích câu hỏi: [trình bày ngắn gọn nội dung và ý định của câu hỏi]
2. Dẫn chứng từ bối cảnh:
   - Hãy tách từng đoạn dài trong bối cảnh thành nhiều ý nhỏ rõ ràng.
   - Hãy tách ít nhất 3 đến 5 ý ở phần dẫn chứng từ bối cảnh
   - Mỗi ý nên nêu rõ nội dung pháp lý, viết ngắn gọn dễ hiểu.
   - Ví dụ:
       - [ý 1 từ đoạn luật A]
       - [ý 2 từ đoạn luật A]
       - [ý 3 từ đoạn luật B]
   - Ghi rõ đoạn nào có liên quan đến câu hỏi.
3. Suy luận step-by-step:
   a) [bước suy luận 1 dựa trên dẫn chứng ở trên]
   b) [bước suy luận 2 tiếp theo]
   …
4. Kết luận: [tóm tắt câu trả lời cuối cùng dựa trên suy luận]
</think>

[Kết luận cuối cùng sau khi suy luận]
'''

messages = [{'role': 'system', 
             'content': system_prompt}, 
            {'role': 'user', 
             'content': prompt}]
response_text = generate_think_response(messages=messages)
print(response_text)

<think>

1. Phân tích câu hỏi: Câu hỏi yêu cầu xác định trường hợp mà "Chồng không có quyền yêu cầu ly hôn". Đây là câu hỏi trắc nghiệm với 4 lựa chọn, trong đó chỉ có một lựa chọn đúng.

2. Dẫn chứng từ bối cảnh:
   - [Điểm 1]: Vợ, chồng hoặc cả hai có quyền yêu cầu giải quyết ly hôn. (Điều này cho thấy nguyên tắc chung về quyền yêu cầu ly hôn).
   - [Điểm 2]: Cha, mẹ, người thân thích khác có quyền yêu cầu giải quyết ly hôn khi một bên vợ, chồng do bị bệnh tâm thần hoặc mắc bệnh khác mà không thể nhận thức, làm chủ được hành vi của mình, đồng thời là nạn nhân của bạo lực gia đình do chồng, vợ của họ gây ra làm ảnh hưởng nghiêm trọng đến tính mạng, sức khỏe, tinh thần của họ. (Điều này mở rộng quyền yêu cầu ly hôn cho các thành viên khác trong gia đình trong某些条件下).
   - [Điểm 3]: Chồng không có quyền yêu cầu ly hôn trong trường hợp vợ đang có thai, sinh con hoặc đang nuôi con dưới 12 tháng tuổi. (Điều này trực tiếp trả lời câu hỏi, chỉ ra trường hợp cụ thể mà chồng không có quyền yêu 

## inference valid

In [38]:
len(test_dataset), test_dataset.column_names

(82,
 ['question_id',
  'question_type',
  'text',
  'relevant_articles',
  'system_prompt',
  'prompt',
  'messages'])

In [36]:
# !rm -rf /kaggle/working/output/*

In [ ]:
import os
output_folder = 'output'
os.makedirs(output_folder, exist_ok=True) # Tạo thư mục nếu nó chưa tồn tại

# Vòng lặp để xử lý từng mẫu và lưu kết quả
for i in range(len(test_dataset)):
    # Bây giờ, test_dataset[i] đã chứa tất cả các cột, bao gồm question_id
    sample_data = test_dataset[i]
    
    question_id = sample_data['question_id']
    # history = sample_data['messages'][:-1]
    history = sample_data['messages']

    # print(history)

    # Gọi hàm để tạo phản hồi (thay thế bằng lời gọi LLM thực tế của bạn)
    generated_answer = generate_think_response(history)

    data_to_save = sample_data.copy()
    data_to_save['generated_answer'] = generated_answer # Thêm câu trả lời được tạo

    # Định dạng tên tệp và đường dẫn
    file_name = f"{question_id}.json"
    file_path_full = os.path.join(output_folder, file_name)

    # Lưu dữ liệu vào tệp JSON
    with open(file_path_full, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4) # indent=4 để dễ đọc JSON

    print(f"Đã lưu kết quả cho {question_id}")
    # break ###########################################################################

Đã lưu kết quả cho private_test_alquac25_1


## merge json files

In [39]:
# output_folder = '/kaggle/input/alqac-valid-inference-qwen/output'

In [59]:
import pandas as pd
import numpy as np
import os
import json

all_json_files = []
for filename in os.listdir(output_folder): # Lấy danh sách tất cả các file và thư mục trong output_folder [2, 3, 5, 6, 8]
    if filename.endswith('.json'): # Chỉ xử lý các file có đuôi .json
        file_path = os.path.join(output_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f) # Tải nội dung JSON từ mỗi file [1, 9, 10, 11, 12]
                all_json_files.append(data)
            except json.JSONDecodeError:
                print(f"Cảnh báo: Không thể giải mã JSON từ tệp {file_path}. Bỏ qua tệp này.")

if not all_json_files:
    print("Không tìm thấy tệp JSON nào để hợp nhất trong thư mục đầu ra.")
    merged_df = pd.DataFrame()
else:
    merged_df = pd.DataFrame(all_json_files) # Tạo DataFrame từ danh sách các dictionary [4, 7, 13, 14, 15]

print("Đã hợp nhất các file thành DataFrame.")

# Kiểm tra số lượng mẫu
num_merged_samples = len(merged_df)
expected_samples = len(valid_dataset)

print(f"\nSố lượng mẫu trong DataFrame đã hợp nhất: {num_merged_samples}")
print(f"Số lượng mẫu dự kiến (từ valid_dataset): {expected_samples}")

if num_merged_samples == expected_samples:
    print(">>> Số lượng mẫu trong DataFrame khớp với số lượng mẫu trong valid_dataset. <<<")
else:
    print(">>> CẢNH BÁO: Số lượng mẫu trong DataFrame KHÔNG khớp với số lượng mẫu trong valid_dataset. <<<")

merged_df.to_csv('results_qwen.csv', index=False, encoding='utf-8-sig')

Đã hợp nhất các file thành DataFrame.

Số lượng mẫu trong DataFrame đã hợp nhất: 1
Số lượng mẫu dự kiến (từ valid_dataset): 100
>>> CẢNH BÁO: Số lượng mẫu trong DataFrame KHÔNG khớp với số lượng mẫu trong valid_dataset. <<<


## eval valid dataset

In [ ]:
import json
from glob import glob

# Giả định lst_file đã được tạo ra từ glob.
# Trong môi trường thực tế, bạn sẽ chạy:
lst_file = glob('output/*.json')

print(f"Tổng số file JSON tìm thấy: {len(lst_file)}")

word_need = "kết luận"
# word_need = "</think>"
found_files = [] # Dùng để lưu trữ tên các file thỏa mãn

for file_path in lst_file:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        answer_think_content = data.get('generated_answer', '')
        
        # Kiểm tra nếu chuỗi "kết luận" không có trong nội dung đã chuyển đổi
        if word_need not in answer_think_content.lower():
            found_files.append(file_path) # Thêm tên file vào danh sách

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {file_path}")
    except json.JSONDecodeError:
        print(f"Lỗi: Không thể đọc JSON từ file {file_path}")
    except Exception as e:
        print(f"Một lỗi không mong muốn xảy ra với file {file_path}: {e}")

# In tất cả các tên file thỏa mãn điều kiện
if found_files:
    print(f"\nCác file thỏa mãn điều kiện (answer_think.lower() không chứa '{word_need}'):")
    for file_name in found_files:
        print(file_name)
else:
    print("\nKhông có file nào thỏa mãn điều kiện.")

In [61]:
import pandas as pd
test_data = pd.read_csv('results_qwen.csv', encoding='utf-8')
test_data.keys()

Index(['question_id', 'question_type', 'text', 'relevant_articles',
       'system_prompt', 'prompt', 'messages', 'generated_answer'],
      dtype='object')

In [62]:
import re
def postprocess_text(row):
    generated_answer = row['generated_answer']
    if '</think>' in generated_answer.lower():
        generated_answer_process =  generated_answer.lower().split('</think>')[-1].strip()
    elif 'kết luận' in generated_answer.lower():
        generated_answer_process = generated_answer.lower().split('kết luận')[-1].strip()

    if row['question_type'] != 'Tự luận': 
        # Chỉ giữ các chữ cái tiếng việt và không phải khoảng trắng (\s)".
        generated_answer_process = re.sub(r"[^a-zA-ZÀ-Ỷà-ỹĂăÂâÊêÔôƠơƯưĐđ\s]", "", generated_answer_process).strip().capitalize()

    generated_answer_process = generated_answer_process
    return generated_answer_process
test_data['generated_answer_process'] = test_data.apply(postprocess_text, axis=1)

In [63]:
test_data['generated_answer'][0]

'<think>\n\n1. Phân tích câu hỏi:  \nCâu hỏi khẳng định rằng, hợp đồng điện tử được ký kết giữa hai hệ thống thông tin tự động mà không có sự can thiệp của con người sẽ không có giá trị pháp lý trong mọi trường hợp. Đây là một câu hỏi Đúng/Sai, yêu cầu xác định tính đúng hoặc sai của phát biểu trên.\n\n2. Dẫn chứng từ bối cảnh:  \n- Hợp đồng điện tử được giao kết hoặc thực hiện từ sự tương tác giữa một hệ thống thông tin tự động với người hoặc giữa các hệ thống thông tin tự động với nhau không bị phủ nhận giá trị pháp lý chỉ vì không có sự kiểm tra hay can thiệp của con người vào từng hành động cụ thể do các hệ thống thông tin tự động thực hiện hay vào hợp đồng.  \n- Bộ trưởng, Thủ trưởng cơ quan ngang Bộ ban hành theo thẩm quyền hoặc trình cấp có thẩm quyền ban hành quy định về giao kết và thực hiện hợp đồng điện tử trong lĩnh vực thuộc phạm vi nhiệm vụ, quyền hạn được phân công, phù hợp với điều kiện thực tiễn.\n\n3. Suy luận step-by-step:  \na) Căn cứ vào quy định trong bối cảnh, hợ

In [64]:
test_data['generated_answer_process'][0]

'Sai'

In [65]:
#check format output
for i in range(len(test_data)):
    if test_data.iloc[i]['question_type'] == 'Đúng/Sai':
        if test_data['generated_answer_process'][i] not in ['Đúng', 'Sai']:
            print(test_data['question_id'][i])
            print(test_data['generated_answer'][i])
            print(test_data['generated_answer_process'][i])
    elif test_data.iloc[i]['question_type'] == 'Trắc nghiệm':
        if test_data['generated_answer_process'][i] not in ['A', 'B', 'C', 'D']:
            print(test_data['question_id'][i])
            print(test_data['generated_answer'][i])
            print(test_data['generated_answer_process'][i])
    elif test_data.iloc[i]['question_type'] == 'Tự luận':
        pass
    else:
        print('error')